To set up and begin running DeepGlycanSite, we first need to create the environment that contains all of its dependencies. I have already established the environment, we just need to run it and make sure its working. 

In [8]:
from pathlib import Path

import docker
import docker.errors
from docker.models.containers import Container

root = Path().resolve() #Root of this local project directory   

try:
    client: docker.client.DockerClient = docker.from_env()
except Exception as e:
    print(e)

client

In [9]:
project_name = "docking_simulation_helpers" #Do Not Change
service_name = "deepglycansite" #Here, choose the docking service you want to use as named in 'docker-compose.yml'

containers = client.containers.list(all=True)

app_container: Container = None

for container in containers:
    if f"{project_name}-{service_name}-1" in container.name:
        print(f"Starting container: {container.name}")
        try:
            container.start()
            print(container.name)
            app_container = container
            break
        except docker.errors.APIError as e:
            print(e)
    

Starting container: docking_simulation_helpers-deepglycansite-1
docking_simulation_helpers-deepglycansite-1


In [ ]:
in_folder = "User" #Change to your name
ligand_sdf = Path(f"{in_folder}" + "/" + "OG29094-gangliotriose.sdf") #Change /folder/example.sdf to the place where your file is in this directory.
print(ligand_sdf)
receptor_pdb = Path(f"{in_folder}" + "/" + "cbmca.pdb") #Change /folder/receptor.pdb the same way
print(receptor_pdb)
# Ensure the input folder exists in the docker container

output_file_name = "cbm3-gd2.txt"

cmd = f"mkdir -p {in_folder}"

app_container.exec_run(cmd)

exit_code, output = app_container.exec_run(cmd)

example_path = Path(str(root) + "/binding_site_files/IQZ2_sites.txt").resolve()
exit_code, output

User/OG29094-gangliotriose.sdf
User/CBM32.pdb


(0, b'')

In [12]:
cmd = f"python3 single_case_prediction.py --conf P2Y14_example/hparams.yaml --ckpt_path ckpts/with_ligand.ckpt --input_fn {receptor_pdb},{ligand_sdf} --out_path User/ --output_fn {output_file_name}"

In [13]:
app_container.exec_run(
    cmd
)

ExecResult(exit_code=1, output=b'fused_multi_tensor is not installed corrected\nfused_rounding is not installed corrected\nfused_layer_norm is not installed corrected\nfused_softmax is not installed corrected\nLoading pretrained weights from /app/DeepGlycanSite/src/unimol_tools/weights/mol_pre_all_h_220816.pt\nUser/OG29094-gangliotriose.sdf\nTraceback (most recent call last):\n  File "/app/DeepGlycanSite/single_case_prediction.py", line 611, in <module>\n    main()\n  File "/app/DeepGlycanSite/single_case_prediction.py", line 543, in main\n    target = prot_to_pyg_graph(rec_file_name, 8.0)\n  File "/app/DeepGlycanSite/single_case_prediction.py", line 291, in prot_to_pyg_graph\n    u = mda.Universe(target_file_name)\n  File "/usr/local/lib/python3.9/site-packages/MDAnalysis/core/universe.py", line 356, in __init__\n    topology = _topology_from_file_like(self.filename,\n  File "/usr/local/lib/python3.9/site-packages/MDAnalysis/core/universe.py", line 118, in _topology_from_file_like\n  